# Detector de Neumonia

In [2]:
# carga de librerias
import tensorflow as tf
print(tf.__version__)
import numpy as np
np.set_printoptions(linewidth=200)
from tensorflow import keras
from keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import zipfile

1.14.0


In [3]:
# definicion de path de archivos
base_dir = 'D:/Documentos/deep_learning/trabajos/Neumonia/chestxray_1'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Directory with our training cat/dog pictures
train_health_dir = os.path.join(train_dir, 'health')
train_ill_vir_dir = os.path.join(train_dir, 'ill_vir')
train_ill_bac_dir = os.path.join(train_dir, 'ill_bac')

# Directory with our validation cat/dog pictures
test_health_dir = os.path.join(test_dir, 'health')
test_ill_vir_dir = os.path.join(test_dir, 'ill_vir')
test_ill_bac_dir = os.path.join(test_dir, 'ill_bac')

In [4]:
# muestro nombres de las imagenes
train_health_fnames = os.listdir(train_health_dir)
train_ill_vir_fnames = os.listdir(train_ill_vir_dir)
train_ill_bac_fnames = os.listdir(train_ill_bac_dir)

print(train_health_fnames[:10])
print(train_ill_vir_fnames[:10])
print(train_ill_bac_fnames[:10])

['IM-0115-0001.jpeg', 'IM-0117-0001.jpeg', 'IM-0119-0001.jpeg', 'IM-0122-0001.jpeg', 'IM-0125-0001.jpeg', 'IM-0127-0001.jpeg', 'IM-0128-0001.jpeg', 'IM-0129-0001.jpeg', 'IM-0131-0001.jpeg', 'IM-0133-0001.jpeg']
['person1000_virus_1681.jpeg', 'person1003_virus_1685.jpeg', 'person1004_virus_1686.jpeg', 'person1005_virus_1688.jpeg', 'person1007_virus_1690.jpeg', 'person1008_virus_1691.jpeg', 'person1009_virus_1694.jpeg', 'person100_virus_184.jpeg', 'person1010_virus_1695.jpeg', 'person1015_virus_1701.jpeg']
['person1000_bacteria_2931.jpeg', 'person1001_bacteria_2932.jpeg', 'person1002_bacteria_2933.jpeg', 'person1003_bacteria_2934.jpeg', 'person1004_bacteria_2935.jpeg', 'person1005_bacteria_2936.jpeg', 'person1006_bacteria_2937.jpeg', 'person1007_bacteria_2938.jpeg', 'person1008_bacteria_2939.jpeg', 'person1010_bacteria_2941.jpeg']


In [5]:
# veo la cantidad de datos
print('total train health images :', len(os.listdir(train_health_dir)))
print('total train ill with virus images :', len(os.listdir(train_ill_vir_dir)))
print('total train ill with bacteria images :', len(os.listdir(train_ill_bac_dir)))

print('total test health images :', len(os.listdir(test_health_dir)))
print('total test ill with virus images :', len(os.listdir(test_ill_vir_dir)))
print('total test ill with bacteria images :', len(os.listdir(test_ill_bac_dir)))

total train health images : 1341
total train ill with virus images : 1345
total train ill with bacteria images : 2530
total test health images : 234
total test ill with virus images : 148
total test ill with bacteria images : 242


In [6]:
# especifico dimensión de las imágenes
img_dim = (128, 128)
batch_size = 500
classes_num = 3

In [13]:
# Genero las imágenes --> vienen vectores de 3 clases
# uso el image generator para train
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_dim,
    batch_size=batch_size,
    class_mode='binary'
)
# uso el image generator para test
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_dim,
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 3 classes.
Found 624 images belonging to 3 classes.


In [8]:
# Modelo 1: NN flattern + FC(relu) + FC(softmax) (3capas)
def modelo_cnn_1 ():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(26, activation='softmax')
    ])
    return model

In [9]:
# creo el modelo 1
model_1 = modelo_cnn_1()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
# compilo el modelo
model_1.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [11]:
# resumen del modelo
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 28800)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               14746112  
_________________________________________________________________
dense_1 (Dense)              (None, 26)                1

In [15]:
# entreno el modelo
history_1 = model_1.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=5,
    validation_data=test_generator,
    validation_steps=50,
    verbose=1
)

Epoch 1/5
100/100 [==============================] - 1562s 16s/step - loss: 0.8200 - acc: 0.6372 - val_loss: 0.6066 - val_acc: 0.7580
Epoch 2/5
100/100 [==============================] - 2665s 27s/step - loss: 0.6945 - acc: 0.7025 - val_loss: 0.6310 - val_acc: 0.7308
Epoch 3/5
 95/100 [===========================>..] - ETA: 1:48 - loss: 0.6637 - acc: 0.7145

KeyboardInterrupt: 